In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [1142]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
#OrderBook class import as written by Kang
import orderbook.orderbook
importlib.reload(orderbook.orderbook)
from orderbook.orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

In [1131]:
#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#            each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kang/Data/TSLA_2015-01-02_34200000_57600000_orderbook_10.csv'

In [1132]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50

In [1133]:
data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2) ##Careful - converts to int32 in jnp array, meaning the filler values get truncated.
newarr=jnp.zeros((int(orderbookLevels*2),7))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200.00000000)


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]

ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 7)


In [1143]:
horizon=91

orderbook_1=JaxOrderBook(price_levels=50,orderQueueLen=50)
trades_init=orderbook_1.process_orders_array(initOB)
trades_msgs=orderbook_1.process_orders_array(ordersJNP[0:horizon])

In [1144]:
MINPRICE=-1.410065407e9
myOB=orderbook_1.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1]).astype(float)
L2OB=L2OB.at[L2OB==MINPRICE].set(-1)
#print(L2OB.reshape(50,4))
#print(myOB.reshape(50,4))
print((myOB.reshape(50,4)-L2OB.reshape(50,4)).astype(int))#L2OB#-myOB

[[       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0        0        0]
 [       0        0      

In [1145]:
ordersJNP[horizon-1]

Array([ 3.000000e+00, -1.000000e+00,  2.000000e+01,  2.257500e+06,
        0.000000e+00,  5.097802e+06,  3.420050e+04], dtype=float32)

## Load the original orderbook implementation on the CPU

In [1146]:
orderbook_2=cpuOrderBook()

In [1147]:
limit_orders = []
order_id_list = [orderbook.orderbook.INITID + i for i in range(100)]
for i in range(50):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
for i in range(50):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

In [1148]:
##Taken largely from the Example.py file in the orderbook repository for the cpu
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype = l1[1] 
    side = 'bid' if l1[5] ==1 else 'ask'
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': ttype,'side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    orderbook_2.processOrder(message,True,False)
    
        
print(message)

{'type': 'delete', 'side': 'ask', 'quantity': 20, 'price': 2257500, 'trade_id': 5097802, 'timestamp': '34200.501657651', 'order_id': 15000009}
{'type': 'delete', 'side': 'ask', 'quantity': 20, 'price': 2257500, 'trade_id': 5097802, 'timestamp': '34200.501657651', 'order_id': 15000009}


In [1149]:
from itertools import zip_longest
cpuOB=jnp.array(list(zip_longest(*orderbook_2.get_L2_state(), fillvalue=-1)))

In [1150]:
print(cpuOB[:50,:])
print(myOB.reshape(50,4))
print((cpuOB[:50,:]-myOB.reshape(50,4).astype('int32')))

[[2229100    1507 2226400     300]
 [2230000      26 2226000     200]
 [2233500      23 2225100      28]
 [2234000      50 2225000     979]
 [2235000     390 2224900       8]
 [2237200     300 2224500      17]
 [2237500     100 2224400       2]
 [2240000     250 2224100     265]
 [2241000     100 2224000      42]
 [2242500      20 2222200     201]
 [2247500      20 2222100      10]
 [2248100      50 2222000       6]
 [2249500     103 2221700      15]
 [2250000     824 2221500     200]
 [2251000     100 2221400      20]
 [2252400     400 2221000      14]
 [2252500      45 2220700     100]
 [2253900     100 2220000    1331]
 [2254500      10 2219800       3]
 [2255000      49 2218300      20]
 [2256000     100 2218000     202]
 [2256900     100 2216800       1]
 [2257500      20 2216300      85]
 [2257800     600 2210000     100]
 [2258800      10 2203200     100]
 [2262700       7 2200000      97]
 [2264900       1 2190000      50]
 [2270000       5 2185100       1]
 [2290000      70 21